**Concept.** Subfigures 1, 2 and 3 in the 1st row, subifugres 4, 5 and 6 in the 2nd row in figure 3. Single colorbar per row.
* subfigure 1 - TE ear model form the plane wave incidence POV;
* subfigure 2 - zoom in to the square-shaped averaging surface;
* subfigure 3 - zoom in to the disk-shaped averaging surface;
* subfigure 4 - TM ear model form the plane wave incidence POV;
* subfigure 5 - zoom in to the square-shaped averaging surface;
* subfigure 6 - zoom in to the disk-shaped averaging surface.

In [ ]:
import os

import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, Circle
import numpy as np
try:
    import open3d as o3d
except ImportError:
    import sys
    print(sys.exc_info())
from scipy import interpolate

from dosipy.utils.dataloader import load_ear_data
from dosipy.utils.integrate import elementwise_dblquad, elementwise_circquad
from dosipy.utils.viz import (set_colorblind, colormap_from_array, scatter_2d,
                              fig_config, save_fig)
from helpers import (ABSTRACT_ID, clean_df, export_pcd, export_fields,
                     poynting_vector, estimate_normals, export_rect_idx,
                     export_circ_idx)

In [ ]:
mode = 'tm'
frequency = 60
df = load_ear_data(mode, frequency)
df = clean_df(df)
xyz = export_pcd(df)
E, H = export_fields(df)
Sx, Sy, Sz = poynting_vector(E, H)

In [ ]:
crop_idxs = np.where(xyz[:, 0] > 67)[0]
xyz_crop = xyz[crop_idxs]
Sx_crop, Sy_crop, Sz_crop = Sx[crop_idxs], Sy[crop_idxs], Sz[crop_idxs]
n_crop = estimate_normals(xyz_crop, knn=30, fast=True)
Sr_crop = abs(Sx_crop.real * n_crop[:, 0]
              + Sy_crop.real * n_crop[:, 1]
              + Sz_crop.real * n_crop[:, 2])
pcd_crop = o3d.geometry.PointCloud()
pcd_crop.points = o3d.utility.Vector3dVector(xyz_crop)
center_crop = pcd_crop.get_center()
pcd_crop.paint_uniform_color([0.5, 0.5, 0.5])
xyz_crop_t = np.c_[xyz_crop[:, 0] - center_crop[0],
                   xyz_crop[:, 1] - center_crop[1],
                   xyz_crop[:, 2] - center_crop[2]]
pcd_crop_t = o3d.geometry.PointCloud()
pcd_crop_t.points = o3d.utility.Vector3dVector(xyz_crop_t)
center_crop_t = pcd_crop_t.get_center()
pcd_crop_t.paint_uniform_color([0.5, 0.5, 0.5])
cframe_crop_t = o3d.geometry.TriangleMesh.create_coordinate_frame(
    size=10, origin=center_crop_t
)
diameter = np.linalg.norm(
    pcd_crop_t.get_max_bound() - pcd_crop_t.get_min_bound()
)
radius = 10 ** 5.5
camera = [0, 0, -diameter]
_, pt_map = pcd_crop_t.hidden_point_removal(camera, radius)
xyz_crop_t_xy = xyz_crop_t[pt_map]
Sr_crop_t_xy = Sr_crop[pt_map]
pcd_crop_t_xy = o3d.geometry.PointCloud()
pcd_crop_t_xy.points = o3d.utility.Vector3dVector(xyz_crop_t_xy)
colors = colormap_from_array(Sr_crop_t_xy)
pcd_crop_t_xy.colors = o3d.utility.Vector3dVector(colors)
avg_center = [0.21, 3.25]
edge_length = 10
area = edge_length ** 2
origin, idx_rect = export_rect_idx(xyz=xyz_crop_t_xy,
                                   center=avg_center,
                                   edge_length=edge_length,
                                   view='xy')
xyz_rect = xyz_crop_t_xy[idx_rect]
Sr_rect = Sr_crop_t_xy[idx_rect]

radius = np.sqrt(area / np.pi)
idx_circ = export_circ_idx(xyz=xyz_crop_t_xy,
                           center=avg_center,
                           radius=radius,
                           view='xy')
xyz_circ = xyz_crop_t_xy[idx_circ]
Sr_circ = Sr_crop_t_xy[idx_circ]

## Subfig 1 and 4

In [ ]:
set_colorblind()
fig_config(latex=True, scaler=1.5, text_size=18)
Sr_label = '$APD$'
fig, ax = scatter_2d({'$x$ [mm]': xyz_crop_t_xy[:, 0],
                      '$y$ [mm]': xyz_crop_t_xy[:, 1],
                      Sr_label: Sr_crop_t_xy}, s=0.1,
                     figsize=(2.950 * 1.5, 2.950 * 1.5))
ax.set_xlim([-20, 20])
patch_rect = Rectangle(origin, edge_length, edge_length, fc='None', lw=2)
patch_circ = Circle(avg_center, radius, fc='None', lw=2)
ax.add_patch(patch_rect)
ax.add_patch(patch_circ)
ax.invert_xaxis()

if mode == 'te':
    subfig_id = 'a'
else:
    subfig_id = 'd'
# fname = os.path.join('figures', f'fig_{ABSTRACT_ID}_3{subfig_id}')
# save_fig(fig, fname=fname, formats=['png'])

## Subfig 2 and 5

In [ ]:
func = interpolate.Rbf(xyz_rect[:, 0], xyz_rect[:, 1], Sr_rect)

x_new = np.linspace(xyz_rect[:, 0].min(), xyz_rect[:, 0].max(), 101)
y_new = np.linspace(xyz_rect[:, 1].min(), xyz_rect[:, 1].max(), 101)
X_new, Y_new = np.meshgrid(x_new, y_new)
Sr_rect_new = func(X_new, Y_new)

In [ ]:
set_colorblind()
fig_config(latex=True, scaler=1.5, text_size=18)
fig = plt.figure()
ax = plt.axes()
ax.imshow(Sr_rect_new, cmap='viridis', origin='lower', interpolation='bicubic')
ax.set(xticks=[0, x_new.size/2, x_new.size-1],
       xticklabels=[round(x_new.min(), 2), round((x_new.min()+x_new.max())/2, 2), round(x_new.max(), 2)],
       yticks=[0, y_new.size/2, y_new.size-1],
       yticklabels=[round(y_new.min(), 2), round((y_new.min()+y_new.max())/2, 2), round(y_new.max(), 2)],
       xlabel='$x$ [mm]')
ax.invert_xaxis()

if mode == 'te':
    subfig_id = 'b'
else:
    subfig_id = 'e'
# fname = os.path.join('figures', f'fig_{ABSTRACT_ID}_3{subfig_id}')
# save_fig(fig, fname=fname, formats=['png'])

In [ ]:
APD_rect = elementwise_dblquad(points=np.c_[xyz_rect[:, 0], xyz_rect[:, 1]],
                               values=Sr_rect,
                               degree=11) / area
APD_rect

## Subfig 3 and 6

In [ ]:
func = interpolate.CloughTocher2DInterpolator(np.c_[xyz_circ[:, 0], xyz_circ[:, 1]], Sr_circ)

x_new = np.linspace(xyz_circ[:, 0].min(), xyz_circ[:, 0].max(), 1001)
y_new = np.linspace(xyz_circ[:, 1].min(), xyz_circ[:, 1].max(), 1001)
X_new, Y_new = np.meshgrid(x_new, y_new)
Sr_circ_new = func(X_new, Y_new)

In [ ]:
set_colorblind()
fig_config(latex=True, scaler=1.5, text_size=18)
fig = plt.figure()
ax = plt.axes()
ax.imshow(Sr_circ_new, cmap='viridis', origin='lower', interpolation='spline36')
ax.set(xticks=[0, x_new.size/2, x_new.size-1],
       xticklabels=[round(x_new.min(), 2), round((x_new.min()+x_new.max())/2, 2), round(x_new.max(), 2)],
       yticks=[0, y_new.size/2, y_new.size-1],
       yticklabels=[round(y_new.min(), 2), round((y_new.min()+y_new.max())/2, 2), round(y_new.max(), 2)],
       xlabel='$x$ [mm]')
ax.invert_xaxis()

if mode == 'te':
    subfig_id = 'c'
else:
    subfig_id = 'f'
# fname = os.path.join('figures', f'fig_{ABSTRACT_ID}_3{subfig_id}')
# save_fig(fig, fname=fname, formats=['png'])

In [ ]:
APD_circ = elementwise_circquad(points=np.c_[xyz_circ[:, 0], xyz_circ[:, 1]],
                                values=Sr_circ,
                                radius=radius,
                                center=avg_center,
                                degree=11) / area
APD_circ